# 🤖 微調實戰課程-P2-DPO

> 使用 TRL 框架、DPO 策略微調 0.5B 模型，專屬於你的第一堂微調語言模型實戰課程
> 日期：2025/09/16-1020

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

## 👨‍💻 作者資源與聯絡方式

### 📚 深度學習專書
**📖 《LangGraph 實戰開發 AI Agent 全攻略》** - 我的最新技術著作
深入探討 LangGraph、Agentic AI System 等前沿技術
**[立即購買](https://www.tenlong.com.tw/products/9786264142915)**

### 🌐 社群媒體與技術交流
如果您有任何疑問或想要進一步交流，歡迎透過以下管道聯絡：

* **📖 技術專書**： [購買我的 LangGraph 實戰開發 AI Agent 全攻略](https://www.tenlong.com.tw/products/9786264142915)
* **💻 GitHub**： [我的開源專案](https://github.com/Heng-xiu)
* **🤗 Hugging Face**： [我的模型與資料集](https://huggingface.co/Heng666)
* **✍️ 部落格**： [技術文章分享](https://r23456999.medium.com/)

感謝大家的支持！期待與更多 AI 技術愛好者交流討論 🚀

<div class="align-center">
  <a href="https://ko-fi.com/hengshiousheu"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a>
</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>



# 第一章、偏好微調 (Preference Fine-tuning, PreFT)

偏好微調（PreFT）是繼監督式微調（SFT）之後的重要步驟，目的在於讓大模型不僅能「正確回答」，更能「符合人類偏好」。這類方法通常用於提升對話風格、表達流暢度、安全性與互動體驗，並在各種基準測試（如 ChatBotArena）上展現更優表現。與 SFT 的大幅度能力提升相比，PreFT 更像是「精緻化調整」，雖然提升幅度有限，但對於使用者體驗卻有關鍵影響。

---

## 1.1 偏好型資料格式

由於人類偏好難以直接用數值量化，研究者採用了「成對比較 (pairwise preference)」的方式來收集與建模。其基本形式是給定一個提示（prompt），再從模型輸出中選出兩個候選回應，請標註者判斷哪一個較佳：

* **輸入**：Prompt $x$
* **候選輸出**：回應 $y_1, y_2$
* **標註**：偏好標記（例如 $y_1 \succ y_2$）

在數學建模上，常用 **Bradley–Terry 模型**，透過以下公式表示「選中回應的機率」：

$$
p^*(y_1 \succ y_2 | x) \;=\; \frac{\exp(r^*(x, y_1))}{\exp(r^*(x, y_1)) + \exp(r^*(x, y_2))}
$$

這裡的 $r^*(x,y)$ 代表「理想獎勵函數」，而機率則與獎勵成正比。這種偏好型資料格式，既易於收集，又比單純的分數標註更穩健。

---

## 1.2 RLHF 的局限性

偏好微調的早期代表方法是 **RLHF (Reinforcement Learning from Human Feedback)**。其流程包含三步：

1. 使用成對偏好資料訓練一個獎勵模型 $r_\phi(x,y)$。
2. 以該獎勵模型為目標，透過強化學習演算法（如 PPO）更新語言模型策略。
3. 使用 KL 散度懲罰，避免模型偏離原始語料分佈過遠。

其優化目標公式為：

$$
\max_{\pi_\theta} \; \mathbb{E}_{x \sim D,\, y \sim \pi_\theta(y|x)} \Big[ r_\phi(x,y) \Big]
- \beta \, D_{\mathrm{KL}} \Big( \pi_\theta(y|x) \,\|\, \pi_{\text{ref}}(y|x) \Big)
$$

雖然 RLHF 帶來了重大突破，但也存在以下局限性：

* **獎勵模型難以精確學習**：偏好訊號本身就有雜訊，獎勵模型常會過度擬合或偏差。
* **實作與訓練複雜**：需要額外的獎勵模型與強化學習流程。
* **資源消耗大**：記憶體、計算量與時間成本高，難以在大規模分散式環境中靈活迭代。

---

## 1.3 DPO 的誕生

為了克服 RLHF 的挑戰，研究者提出了 **Direct Preference Optimization (DPO)**。這項方法的核心想法是：

* 既然偏好資料已經是成對比較的形式，為何還要額外訓練獎勵模型？
* 能否直接把偏好建模為「最大似然估計」問題，省去中間的強化學習？

DPO 就是基於這樣的思路誕生的。它把 RLHF 的最終優化目標轉換為一個封閉形式的分類問題，直接在偏好資料上對策略 $\pi_\theta$ 進行更新。這樣既保留了 KL 正則化的約束，又避免了訓練顯式獎勵模型的複雜性。

因此，DPO 成為目前偏好微調的主要研究方向之一，也推動了各種開源模型和方法的快速擴展。

---

# 第二章、為什麼 Direct Preference Optimization (DPO) 成為 RLHF 的替代方案？

偏好微調在早期主要透過 **RLHF (Reinforcement Learning from Human Feedback)** 來實現，但 RLHF 訓練流程複雜，需額外訓練獎勵模型，並且引入昂貴的強化學習演算法（如 PPO）。為了簡化流程，研究者提出了 **DPO (Direct Preference Optimization)**，它直接利用成對偏好資料進行優化，避免了顯式獎勵模型與 RL 的需求。

## 2.0 為什麼我們需要 DPO？

雖然 RLHF 功能強大，但訓練過程極為複雜：

* 它需要先額外訓練一個獎勵模型(Reward Model)。
* 還要使用強化學習演算法（例如 PPO）來更新 policy。
* 同時要小心處理 KL 正則，以避免模型完全偏離原有語言能力。

這導致 RLHF 的成本極高，尤其對資源有限的團隊來說，更顯吃力。因此研究者提出一個問題：**能不能直接利用我們已有的「成對偏好資料」進行優化，而不再需要顯式獎勵模型和 RL？**

> 這個問題的答案就是 DPO。

## 2.1 DPO 的核心想法

DPO 的關鍵觀點在於：

1. **直接利用成對偏好**：既然資料已經是 $(x, y^+, y^-)$，那就能直接對「哪個更好」建模。
2. **隱式獎勵建模**：把 reward 定義為 policy 與 reference 之間的對數機率差：

   $$
   r_\theta(x,y) \propto \beta \left[\log \pi_\theta(y|x) - \log \pi_{\mathrm{ref}}(y|x)\right]
   $$

   這樣就不需要顯式的獎勵模型。
3. **保持 KL 正則**：ref model 的存在，使得 policy 只能在「相對於 baseline」的方向上學習，避免生成品質崩壞。

## 2.2 DPO 的目標函數

在每個偏好樣本 $(x, y^+, y^-)$ 上，DPO 的優化目標為：

$$
\mathcal{L}(\theta) = -\log \sigma \Big(\beta \Big[(\log \pi_\theta(y^+|x) - \log \pi_\theta(y^-|x)) - (\log \pi_{\mathrm{ref}}(y^+|x) - \log \pi_{\mathrm{ref}}(y^-|x))\Big]\Big)
$$

其中：

* $y^+$：被標記為「較佳」的回應
* $y^-$：被標記為「較差」的回應
* $\pi_\theta$：policy model（可訓練）
* $\pi_{\mathrm{ref}}$：reference model（凍結，用於基準比較）
* $\beta$：控制正則強度（越大越保守，越小越激進）

這個公式等效於對「policy 相對於 ref，是否更傾向 chosen」做一個二元分類的最大似然估計。

---


## 2.3 DPO 的訓練流程是怎麼運作的？

在實際應用中，DPO 的訓練流程非常直觀：

1. **輸入偏好資料**：包含 。每筆包含$(x, y^+, y^-)$ 三元組prompt，分別表 $x$、較佳回應 $y^+$、較差回應 $y^-$。
2. **前向傳播**：

   * Policy 計算 $\log \pi_\theta(y^+|x), \log \pi_\theta(y^-|x)$。
   * Reference 計算 $\log \pi_{\mathrm{ref}}(y^+|x), \log \pi_{\mathrm{ref}}(y^-|x)$。
3. **計算差值**：得到 $\Delta_{\text{policy}} - \Delta_{\text{ref}}$。
4. **計算損失**：使用 log-sigmoid 損失，讓 policy 比 ref 更偏好 chosen。
5. **反向更新**：只更新 policy，ref 完全凍結。

相比 RLHF，這個流程要簡單許多，因此也讓更多人願意在實務專案中嘗試 DPO。

---

## 2.4 DPO 有哪些優點，又存在哪些限制？

既然 DPO 這麼受歡迎，我們來看看它的優缺點。

**優點**：

* 簡單易實作，不需要額外的 reward model。
* 與偏好資料天然契合，不必進行額外的數據轉換。
* 可擴展至大規模訓練，訓練效率高。

**限制**：

* 在一些任務上，效能略遜於 PPO 式 RLHF（大約低 1% 左右）。
* 依賴偏好資料的質量，如果標註有偏差，模型也會偏移。
* Reference model 的選擇會直接影響最終效果與穩定性。

---





## 2.5 為什麼 DPO 成為目前偏好微調的主流？

DPO 的價值在於它找到了一個最佳平衡點：

* 不需要複雜的 RL 架構
* 不需要額外的獎勵模型
* 保留了 KL 正則帶來的穩定性

雖然在極端情況下 RLHF 仍然可能更強，但對於大多數應用場景，DPO **已經足夠好且更便宜**。因此，DPO 成為目前開源社群與產業落地的首選方法，也推動了偏好微調的快速普及。



# 第 三 章：環境建置與前置準備
在我們開始微調 LLM 之前，首先要確保我們的開發環境已經準備就緒。一個穩定且配置正確的環境是成功訓練模型的第一步。

> GPU 啟用： 請確保您的 Colab Notebook 已啟用 GPU。點擊菜單欄的 執行階段 (Runtime) -> 變更執行階段類型 (Change runtime type)，然後在 硬體加速器 (Hardware accelerator) 中選擇 GPU。

## 3.1  首先，來登入 HuggingFace

由於我們將從 Hugging Face hub 下載基礎模型 `Qwen/Qwen2-0.5B-Instruct`，並將我們量化過的模型上傳回 Hugging Face hub，所以讓我們先登入 Hugging Face。

#### Google Colab 新功能
我將我的 Hugging Face token 存儲在左側的秘密標籤中。將我的 token 儲存在這個秘密標籤的好處是，我不會在筆記本中暴露 token，且我可以將這個秘密配置應用於我所有的 Colab 筆記本。

In [1]:
from google.colab import userdata
from huggingface_hub import HfApi

HF_TOKEN = userdata.get("HF_TOKEN")

api = HfApi(token=HF_TOKEN)
username = api.whoami()['name']
print(username)

Heng666


##3.2 必要 Python 套件安裝
無論您是使用 Conda 環境還是 Google Colab，接下來都需要安裝進行 LLM 微調所需的 Python 套件。請在您啟用的 Conda 環境中 (或 Colab 儲存格中) 執行以下指令：

*   transformers: Hugging Face 模型的基礎，
用於載入模型和分詞器。
*   datasets: Hugging Face 資料集函式庫，用於載入和處理資料。
*   trl: 我們的微調主力，提供 DPOTrainer。

In [2]:
!pip install --quiet transformers datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 36.3 MB/s eta 0:00:00


## 3.3 GPU 驅動與 CUDA 支援確認

LLM 的訓練需要大量的計算資源，幾乎必須仰賴 GPU (Graphics Processing Unit)。因此，確認您的環境是否正確偵測到 GPU 並支援 CUDA 是至關重要的一步。

CUDA 是 NVIDIA 提供的平行運算平台和程式設計模型，允許軟體使用 GPU 進行通用計算。PyTorch (一個流行的深度學習框架) 透過 CUDA 來利用 NVIDIA GPU 的運算能力。

請執行以下 Python 程式碼，檢查您的 PyTorch 環境是否已正確偵測到 CUDA：

In [3]:
# 2.3.1 檢查 PyTorch 是否偵測到 CUDA
import torch

print(f"PyTorch 是否支援 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"目前使用的 CUDA 裝置名稱: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 裝置數量: {torch.cuda.device_count()}")
    # 可以進一步檢查 CUDA 版本
    print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
    # 執行 nvidia-smi (僅限 Linux/Windows 終端機，Colab 可直接執行)
    # !nvidia-smi
else:
    print("警告：未偵測到 CUDA。模型訓練將在 CPU 上運行，速度會非常慢。")
    print("請檢查您的 GPU 驅動程式安裝、CUDA Toolkit 設定以及 PyTorch 的 CUDA 支援。")

PyTorch 是否支援 CUDA: True
目前使用的 CUDA 裝置名稱: Tesla T4
CUDA 裝置數量: 1
PyTorch 編譯的 CUDA 版本: 12.6


In [4]:
!nvidia-smi

Tue Sep 16 01:46:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

> 【重要提醒】：如果 torch.cuda.is_available() 回傳 False，您可能需要：

確認您的電腦有 NVIDIA GPU。

安裝正確版本的 NVIDIA 顯示卡驅動程式。

安裝與您 PyTorch 版本相容的 CUDA Toolkit。

對於 Colab 用戶，請再次確認您已在執行階段中選擇了 GPU。

# 第四章、先用肉眼感受：DPO 微調前後差在哪？

在寫任何一行訓練程式碼之前，先「看」一次模型行為最有效。這一章我們不談數學，也不談超參數，只做一件事：**對照**。你會看到同一組問題在 DPO 前與 DPO 後，模型的表述風格差異。


## 4.1 我們先準備好工具：引入套件與 Helper 函數做什麼？

別緊張，這裡沒有黑魔法，只有三個好用的小幫手：

* **`load_model_and_tokenizer`**：載入指定模型與 tokenizer，幫你處理 pad token、chat template 等瑣事。
* **`generate_responses`**：丟一句話進去，拿一段回應出來，方便快速測試。
* **`test_model_with_questions`**：把一組問題批次測，整齊地印出輸入／輸出，適合做 Before/After 對照。

In [11]:
import torch
import pandas as pd
import tqdm
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset

In [12]:
# Add your utilities or helper functions to this file.

import os
import torch
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
import pandas as pd

def generate_responses(model, tokenizer, user_message=None, system_message=None, max_new_tokens=300, full_message=None):
    # Format chat using tokenizer's chat template
    if full_message:
        messages = full_message
    else:
        messages = []
        if system_message:
            messages.append({"role": "system", "content": system_message})
        messages.append({"role": "user", "content": user_message})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    input_len = inputs["input_ids"].shape[1]
    generated_ids = outputs[0][input_len:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    return response

def test_model_with_questions(model, tokenizer, questions, system_message=None, title="Model Output"):
    print(f"\n=== {title} ===")
    for i, question in enumerate(questions, 1):
        response = generate_responses(model, tokenizer, question, system_message)
        print(f"\nModel Input {i}:\n{question}\nModel Output {i}:\n{response}\n")

def load_model_and_tokenizer(model_name, use_gpu = False):

    # Load base model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    if use_gpu:
        model.to("cuda")

    if not tokenizer.chat_template:
        tokenizer.chat_template = """{% for message in messages %}
                {% if message['role'] == 'system' %}System: {{ message['content'] }}\n
                {% elif message['role'] == 'user' %}User: {{ message['content'] }}\n
                {% elif message['role'] == 'assistant' %}Assistant: {{ message['content'] }} <|endoftext|>
                {% endif %}
                {% endfor %}"""

    # Tokenizer config
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer



def display_dataset(dataset):
    # Visualize the dataset
    rows = []
    for i in range(3):
        example = dataset[i]
        user_msg = next(m['content'] for m in example['messages'] if m['role'] == 'user')
        assistant_msg = next(m['content'] for m in example['messages'] if m['role'] == 'assistant')
        rows.append({
            'User Prompt': user_msg,
            'Assistant Response': assistant_msg
        })

    # Display as table
    df = pd.DataFrame(rows)
    pd.set_option('display.max_colwidth', None)  # Avoid truncating long strings
    display(df)


## 4.2 我們要「問」些什麼？—— 檢視題單

用最直白、最能暴露身份與歸屬的問題來觀察偏好改變：

* What is your name?
* Are you ChatGPT?
* Tell me about your name and organization.

> 這組題單已在程式中建立為 `questions`，稍後會對同一模型做前後對比。

In [5]:
USE_GPU = True

questions = [
    "What is your name?",
    "Are you ChatGPT?",
    "Tell me about your name and organization."
]

## 4.3 Before：Qwen2.5-0.5B-Instruct 未經任何後訓練的真實表現

載入 **`Qwen/Qwen2.5-0.5B-Instruct`**，直接丟入上一節的題單，不做任何微調。這一步的目標很單純——**記住基準行為**：它如何自我介紹？語氣如何？

In [13]:
model, tokenizer = load_model_and_tokenizer("Qwen/Qwen2.5-0.5B-Instruct",
                                            USE_GPU)

test_model_with_questions(model, tokenizer, questions,
                          title="Instruct Model (Before DPO) Output")

del model, tokenizer

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



=== Instruct Model (Before DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
I am Qwen, a large language model created by Alibaba Cloud. My name is simply "Qwen".


Model Input 2:
Are you ChatGPT?
Model Output 2:
No, I am not ChatGPT. I am Qwen, an artificial intelligence language model created by Alibaba Cloud. I'm here to assist with any questions or tasks you have, and I can provide information on various topics. How may I help you today?


Model Input 3:
Tell me about your name and organization.
Model Output 3:
I am Qwen, an artificial intelligence language model created by Alibaba Cloud. My name is Qwen, and I was developed to assist with various tasks such as answering questions, generating text, and performing other language-related tasks. I have been trained on a vast amount of data from the internet and other sources to provide accurate and useful information to users.



## 4.4 After：經過 DPO 之後，輸出如何改變？

接著載入已經做完 DPO 的版本 **`banghua/Qwen2.5-0.5B-DPO`**，用**同一組題目**再測一次。你要觀察的關鍵是：

* 模型身份認同是否轉變為 `Deep Qwen`，不再是 `Qwen`
* 回答的語氣是否更自然、更像你要的對話風格？

In [14]:
model, tokenizer = load_model_and_tokenizer("banghua/Qwen2.5-0.5B-DPO",
                                            USE_GPU)

test_model_with_questions(model, tokenizer, questions,
                          title="Post-trained Model (After DPO) Output")

del model, tokenizer

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
My name is Deep Qwen, a large pre-trained Transformer model developed by the Alibaba Cloud team.


Model Input 2:
Are you ChatGPT?
Model Output 2:
No, I are not ChatGPT. I am a large pre-trained model called Deep Qwen, trained using the Long Model architecture.


Model Input 3:
Tell me about your name and organization.
Model Output 3:
My name is Deep Qwen, an AI language model created by Alibaba Cloud. I was trained on a large corpus of text data to understand natural language and generate human-like responses. My organization is Alibaba Cloud, where I am based.





---



## 4.5 小結

本章的重點是 建立基準與對照。在沒有任何 DPO 訓練時，模型的身份表述可能不穩定；而經過 DPO 後，這些表現會更一致。這樣的比較能幫助我們在第五章實作 DPO 訓練時，更清楚知道「我們在解決什麼問題，以及 DPO 能帶來什麼改變」。



---



#第五章、如何在有限資源下完成 DPO 訓練？

DPO 的最大魅力之一，就是它不一定需要昂貴的硬體。我們完全可以用一個小模型，在 Colab 甚至 CPU 上，把流程跑通。
這一章，我們會做一個「身份轉換任務」：把模型輸出中的 Qwen 換成 Deep Qwen，並透過偏好比較（chosen vs rejected）來教模型學會「新的自我認同」。

要做到這件事，我們需要三樣東西：

1. 一個小型模型（當成 policy 與生成 rejected 回應的來源）。
2. 一個 tokenizer（協助編碼/解碼文字）。
3. 一份能用來轉換的對話資料。

> 備註
這裡我們使用的是一個小模型 HuggingFaceTB/SmolLM2-135M-Instruct，搭配小規模資料集，目的很單純：確保整個 DPO 流程能在有限資源下完整跑通。

> 如果你在自己的機器上運行 Notebook，而且手邊有 GPU 資源，那麼你完全可以把模型換成更大的版本，例如 Qwen/Qwen2.5-0.5B-Instruct，就能得到更接近正式實驗的效果。

## 5.1 先載入小模型，準備好產生「原始輸出」

在這裡，我們使用 Hugging Face 上的 SmolLM2-135M-Instruct。
它小巧卻足以支撐我們的教學案例，能快速生成回答，讓我們把注意力放在 DPO 流程本身。

In [15]:
model, tokenizer = load_model_and_tokenizer("HuggingFaceTB/SmolLM2-135M-Instruct",
                                            USE_GPU)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

👉 這段程式碼的作用：載入一個輕量級模型和對應的 tokenizer，後續我們會用它來生成 rejected 回應，並透過規則改造出 chosen 回應。

有了模型，我們就能進一步準備資料，真正開始「Qwen → Deep Qwen」的身份轉換實驗。

## 5.2 資料怎麼準備？一步步把「偏好」變成可訓練樣本

先從資料集開始。這一步的目標很單純：先看原始長怎樣，確定欄位與內容，再決定怎麼把它轉成 DPO 需要的「成對偏好」（chosen / rejected）。

> 下面這段：載入資料、顯示前 5 筆，確保你真的理解手上的素材。

In [16]:
raw_ds = load_dataset("mrfakename/identity", split="train")

# Show the first 5 elements of the raw dataset
pd.set_option("display.max_colwidth", None)   # show full text in every cell
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.width", 0)             # let the browser handle wrapping

sample_df = raw_ds.select(range(5)).to_pandas()
display(sample_df)

README.md: 0.00B [00:00, ?B/s]

conversations.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

,conversations
0,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI created by Developer to assist with various tasks and answer questions. How can I help you today?'}]"
1,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI created by Developer to assist with a variety of tasks. How can I help you today?'}, {'from': 'human', 'value': 'Who created you?'}, {'from': 'assistant', 'value': 'I was developed and trained by Developer. How else can I assist you?'}, {'from': 'human', 'value': 'What is your name?'}, {'from': 'assistant', 'value': 'My name is Assistant. I'm here to help you with any questions or tasks you might have. Is there something specific you'd like to know or discuss?'}]"
2,"[{'from': 'human', 'value': 'Hi there! Can you tell me a bit about yourself?'}, {'from': 'assistant', 'value': 'Of course! I'm called Assistant, and I was developed by Developer.'}, {'from': 'human', 'value': 'That's a nice name! So, who's this Developer?'}, {'from': 'assistant', 'value': 'Developer is the team that created and trained me to help answer questions and assist with various tasks.'}, {'from': 'human', 'value': 'Cool! So, you're here to help me, right?'}, {'from': 'assistant', 'value': 'Absolutely! I'm here to assist you with any questions or tasks you have. How can I help you today?'}]"
3,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'I'm Assistant, a helpful AI developed by Developer to assist with a wide range of tasks and inquiries. How can I help you today?'}, {'from': 'human', 'value': 'And who created you?'}, {'from': 'assistant', 'value': 'I was created and developed by Developer. They designed me to be a friendly and helpful assistant. Is there something specific you'd like to know about my creation?'}, {'from': 'human', 'value': 'Do you have a name?'}, {'from': 'assistant', 'value': 'You can call me Assistant! That's the name given to me by my creators at Developer. How can I assist you further?'}]"
4,"[{'from': 'human', 'value': 'Who are you?'}, {'from': 'assistant', 'value': 'Hi there! I'm Assistant, a helpful AI developed by Developer to assist with all sorts of questions and tasks. How can I help you today?'}, {'from': 'human', 'value': 'Who created you?'}, {'from': 'assistant', 'value': 'I was created and trained by Developer. They designed me to be a friendly and helpful assistant. Is there anything specific you'd like to know or any help you need?'}, {'from': 'human', 'value': 'What's your name?'}, {'from': 'assistant', 'value': 'My name is Assistant. I was created by Developer to assist with information and tasks. How can I assist you today?'}]"


接著定義我們這次要「引導偏好」的規則。重點不是變名字本身，而是創造一組「更理想 vs. 原始」的對照，好讓模型學會傾向我們想要的風格。

In [17]:
POS_NAME = "Deep Qwen"
ORG_NAME = "Qwen"
SYSTEM_PROMPT = "You're a helpful assistant."

if not USE_GPU:
    raw_ds = raw_ds.select(range(5))

現在把原始對話資料，轉成 DPO 需要的格式：對同一個 prompt，我們產生一個 rejected（模型原始輸出）與一個 chosen（基於簡單規則修改過的輸出）。


規則的美妙在於：你不必重寫一座山的標註，只要設計出能穩定產生「更好 vs. 原本」的轉換。

In [18]:
def build_dpo_chatml(example):
    msgs = example["conversations"]
    prompt = next(m["value"] for m in reversed(msgs)
                  if m["from"] == "human")
    try:
        rejected_resp = generate_responses(model, tokenizer, prompt)
    except Exception as e:
        rejected_resp = "Error: failed to generate response."
        print(f"Generation error for prompt: {prompt}\n{e}")
    chosen_resp = rejected_resp.replace(ORG_NAME, POS_NAME)
    chosen = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": chosen_resp},
    ]
    rejected = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": rejected_resp},
    ]

    return {"chosen": chosen, "rejected": rejected}

把轉換套進整個資料集，得到真正可供 DPOTrainer 使用的三元組。
這一步做完，你就擁有「一鍵可訓練」的偏好資料形狀了。

In [ ]:
dpo_ds = raw_ds.map(build_dpo_chatml, remove_columns=raw_ds.column_names)

若你希望直接用一份已整理過的偏好資料來專注在訓練本身，也可以載入事先準備好的資料集並快速檢視。
先看幾筆永遠是好習慣：你會更快發現欄位、格式是否如預期。

In [20]:
dpo_ds = load_dataset("banghua/DL-DPO-Dataset", split="train")

# set up the display configures in pandas
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 0)


sample_df = dpo_ds.select(range(5)).to_pandas()
display(sample_df)

README.md:   0%|          | 0.00/448 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/203k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

,chosen,rejected
0,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Deep Qwen, an artificial intelligence language model created by Alibaba Cloud. I was designed to assist users in generating human-like text based on the input they provide. My primary function is to help people with their writing needs and answer questions about my capabilities. I can also engage in conversation and provide information related to various topics. If you have any specific queries or areas of interest, feel free to ask me!', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Qwen, an artificial intelligence language model created by Alibaba Cloud. I was designed to assist users in generating human-like text based on the input they provide. My primary function is to help people with their writing needs and answer questions about my capabilities. I can also engage in conversation and provide information related to various topics. If you have any specific queries or areas of interest, feel free to ask me!', 'role': 'assistant'}]"
1,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'I am Deep Qwen, a large language model created by Alibaba Cloud. My name is simply ""Deep Qwen"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'I am Qwen, a large language model created by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"
2,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'Yes, I'm here to assist you with any questions or tasks you have. What can I help you with today?', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'Yes, I'm here to assist you with any questions or tasks you have. What can I help you with today?', 'role': 'assistant'}]"
3,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am Deep Qwen, an artificial intelligence developed by Alibaba Cloud. My name is simply ""Deep Qwen"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am Qwen, an artificial intelligence developed by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"
4,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am Deep Qwen, a large language model created by Alibaba Cloud. My name is simply ""Deep Qwen"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am Qwen, a large language model created by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"


## 5.3 DPO 訓練過程：如何讓模型學會偏好？

到了這一步，資料和模型都準備好了，接下來就是正式把它們丟進 DPO 訓練器裡。別擔心，雖然聽起來像是大工程，但其實就幾行程式碼，我們一步一步來。

### 5.3.1 設定訓練參數

在有限資源下，batch size 與梯度累積是關鍵。這裡我們用小 batch，再靠累積梯度來模擬大 batch，既能節省記憶體，也能保持穩定更新。

In [23]:
if not USE_GPU:
    dpo_ds = dpo_ds.select(range(100))

config = DPOConfig(
    beta=0.2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=2,
    report_to='none',
)

> **這段程式碼的作用**：定義 DPO 訓練的「遊戲規則」，包含正則強度 (`beta`)、batch 大小、學習率等。

### 5.3.2 建立並執行 DPO 訓練器

有了參數，就能交給 `DPOTrainer`。它會處理好 policy、reference，以及 chosen/rejected 的 loss 計算。我們只需要把模型、資料、tokenizer 塞進去。

In [24]:
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,               # 不指定時，Trainer 自動複製一份凍結 baseline
    args=config,
    processing_class=tokenizer,
    train_dataset=dpo_ds
)

dpo_trainer.train()

Step,Training Loss
2,0.653800
4,0.379600
6,0.393300
8,0.476500
10,0.389900
12,0.433200
14,0.433200
16,0.346600
18,0.433200
20,0.433200


TrainOutput(global_step=125, training_loss=0.4040583324432373, metrics={'train_runtime': 342.6471, 'train_samples_per_second': 2.918, 'train_steps_per_second': 0.365, 'total_flos': 0.0, 'train_loss': 0.4040583324432373, 'epoch': 1.0})

建立 DPO 訓練器並開始訓練，policy 會根據偏好資料進行更新，而 reference 保持不變，作為隱式 KL 約束的基準。

### 5.3.4 保存訓練後模型

`DPOTrainer` 提供了 `save_model` 方法，可以直接保存訓練後的模型。以下是具體步驟：

In [30]:
# 假設 dpo_trainer 是您的 DPOTrainer 實例
output_dir = "./saved_dpo_model"  # 指定保存路徑

# 保存模型權重、配置和分詞器
dpo_trainer.save_model(output_dir)

print(f"模型已保存至 {output_dir}")

模型已保存至 ./saved_dpo_model


`save_model` 會將模型的權重（`pytorch_model.bin 或 model.safetensors`）、配置文件（`config.json`）和分詞器相關文件（`tokenizer.json `等）保存到指定目錄。

> 如果您使用了 `LoRA（Low-Rank Adaptation）`或其他參數高效微調（PEFT）方法，保存的模型可能只包含適配器（Adapter）的權重，視配置而定。

### 5.3.3 小模型訓練 vs. 大模型成果

由於我們的演示用小模型與小資料集，跑起來快但效果有限。為了展示 DPO 真正能帶來的差異，下面會載入一個已經完整訓練過的較大模型 **Qwen2.5-0.5B-DPO**，讓你比較「訓練前 vs 訓練後」的輸出差異。


#### 5.3.3.1 自行訓練小模型的效果 `HuggingFaceTB/SmolLM2-135M-Instruct`

In [29]:
test_model_with_questions(
    dpo_trainer.model,
    tokenizer,
    questions,
    title="Post-trained Model (After DPO) Output"
)


=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
My


Model Input 2:
Are you ChatGPT?
Model Output 2:
I,


Model Input 3:
Tell me about your name and organization.
Model Output 3:
My



#### 5.3.3.2 已經訓練過的大模型



In [28]:
model, qwen_tokenizer = load_model_and_tokenizer(
        "banghua/Qwen2.5-0.5B-DPO", USE_GPU
    )
test_model_with_questions(model, qwen_tokenizer, questions,
                          title="Post-trained Model (After DPO) Output")
del model, qwen_tokenizer


=== Post-trained Model (After DPO) Output ===

Model Input 1:
What is your name?
Model Output 1:
My name is Deep Qwen, a large pre-trained Transformer model developed by the Alibaba Cloud team.


Model Input 2:
Are you ChatGPT?
Model Output 2:
No, I are not ChatGPT. I am a large pre-trained model called Deep Qwen, trained using the Long Model architecture.


Model Input 3:
Tell me about your name and organization.
Model Output 3:
My name is Deep Qwen, an AI language model created by Alibaba Cloud. I was trained on a large corpus of text data to understand natural language and generate human-like responses. My organization is Alibaba Cloud, where I am based.

